In [1]:
import torch
import pandas as pd
import numpy as np

# Add the prime_torch file to the system path so we can import it
import sys
sys.path.append("/glade/u/home/cobrien/prime/prime_lib/primesw")
from data import SWDataset, SWDataModule

In [2]:
test_dataframe = pd.DataFrame([])
test_dataframe['time'] = pd.date_range(
    pd.to_datetime('20150902 00:00:00+0000'),
    pd.to_datetime('20250101 00:00:00+0000'),
    freq = '100s'
)
test_dataframe['a'] = np.arange(len(test_dataframe)) # Fake input
test_dataframe['b'] = test_dataframe['a'] * 2 # Fake target
test_dataframe['pos'] = test_dataframe['a'] * 0.5 # Fake positions
test_dataframe.to_hdf("~/data/prime/test.h5", key = 'lineartest')

In [3]:
datamodule = SWDataModule(
    target_features = ['b'],
    input_features = ['a'],
    position_features = ['pos'],
    freq = '100s',
    window = 100,
    stride = 10,
    interp_frac = 0.1,
    datastore = "~/data/prime/test.h5",
    key = "lineartest",
)
datamodule.setup()

2025-09-03 13:18:12.885 | INFO     | data:setup:219 - Train dataloader is ready. Dataset size: 1769473
2025-09-03 13:19:01.638 | INFO     | data:setup:237 - Validation dataloader is ready. Dataset size: 589249
2025-09-03 13:19:50.478 | INFO     | data:setup:255 - Test dataloader is ready. Dataset size: 586657


In [5]:
import torch
import torch.nn as nn

In [6]:
hidden_layers = [10,20,30]
total_input_dim = 10
tar_dim = 5
p_drop = 0.2
network = nn.Sequential( #Make the starting layer
    nn.Linear(total_input_dim, hidden_layers[0]),
    nn.ReLU(),
    nn.Dropout(p=p_drop),
)

In [11]:
datamodule.trn_ds.input_data.shape

torch.Size([1769364, 100, 1])